<a href="https://colab.research.google.com/github/jyoti-chn/Automated-Prediction-of-Item-Difficulty-and-Item-Response-Time/blob/main/Response_Time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
# drive.mount('/content/gdrive/', force_remount=True)
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
train_file = '/content/gdrive/MyDrive/ML-Project/train_final.csv'
test_file = '/content/gdrive/MyDrive/ML-Project/test_final.csv'
gold_file = '/content/gdrive/MyDrive/ML-Project/gold_final.csv'

In [ ]:
import pandas as pd

train_final_df = pd.read_csv(train_file)


print(train_final_df.describe())

          ItemNum  Difficulty  Response_Time
count  466.000000  466.000000     466.000000
mean   330.444206    0.482532      85.555343
std    191.249064    0.307830      28.588745
min      2.000000    0.020000      26.960000
25%    170.250000    0.240000      64.727500
50%    324.500000    0.420000      80.490000
75%    494.750000    0.700000     101.642500
max    666.000000    1.380000     229.430000


In [ ]:
pip install transformers pandas torch textstat nltk

In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [ ]:
import pandas as pd
import numpy as np
import torch
torch.cuda.empty_cache()
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import textstat
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def preprocess_text(text):
    # text readability scores (Flesch Reading Ease)
    readability_score = textstat.flesch_reading_ease(text)
    return readability_score

train_final_df['readability_score'] = train_final_df['ItemStem_Text'].apply(preprocess_text)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# def encode_data(tokenizer, questions, answers_text, correct_answers, max_length=512):
#     input_ids_list = []
#     attention_masks_list = []
#     for question, answers, correct_answer in zip(questions, answers_text, correct_answers):
#         input_ids = []
#         attention_masks = []
#         # Encode question text
#         encoded_dict = tokenizer.encode_plus(question, add_special_tokens=True, max_length=max_length,
#                                              pad_to_max_length=True, return_attention_mask=True, return_tensors='pt')
#         input_ids.append(encoded_dict['input_ids'])
#         attention_masks.append(encoded_dict['attention_mask'])

#         # Encode concatenated answer options text
#         encoded_dict = tokenizer.encode_plus(answers, add_special_tokens=True, max_length=max_length,
#                                              pad_to_max_length=True, return_attention_mask=True, return_tensors='pt')
#         input_ids.append(encoded_dict['input_ids'])
#         attention_masks.append(encoded_dict['attention_mask'])

#         # Encode correct answer text
#         encoded_dict = tokenizer.encode_plus(correct_answer, add_special_tokens=True, max_length=max_length,
#                                              pad_to_max_length=True, return_attention_mask=True, return_tensors='pt')
#         input_ids.append(encoded_dict['input_ids'])
#         attention_masks.append(encoded_dict['attention_mask'])

#         input_ids_list.append(torch.cat(input_ids, dim=0))
#         attention_masks_list.append(torch.cat(attention_masks, dim=0))

#     return torch.stack(input_ids_list, dim=0), torch.stack(attention_masks_list, dim=0)

def encode_data(tokenizer, texts, max_length=512):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoded_dict = tokenizer.encode_plus(text, add_special_tokens=True, max_length=max_length,
                                              pad_to_max_length=True, return_attention_mask=True, return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

# Replace NaN values with empty strings in answer columns to avoid NaN in combined text
answer_columns = ['Answer__A', 'Answer__B', 'Answer__C', 'Answer__D', 'Answer__E',
                  'Answer__F', 'Answer__G', 'Answer__H', 'Answer__I', 'Answer__J']
train_final_df[answer_columns] = ' ' + train_final_df[answer_columns].fillna('')

# Combine itemtext and answer columns into 'combined_text' column
train_final_df['combined_answer_text'] = train_final_df[answer_columns].agg(' '.join, axis=1)
train_final_df['combined_answer_text'] = train_final_df['combined_answer_text'].astype(str)
# print(train_final_df.loc[0, 'combined_text'])

train_final_df['Answer_Text'] = train_final_df['Answer_Text'].fillna('')

# Convert 'type' column to numerical values
type_mapping = {'Text': 0, 'PIX': 1}
train_final_df['type_numeric'] = train_final_df['ItemType'].replace(type_mapping).astype(int)

# Convert 'exam' column to numerical values
exam_mapping = {'STEP 1': 0, 'STEP 2': 1, 'STEP 3': 2}
train_final_df['exam_numeric'] = train_final_df['EXAM'].replace(exam_mapping).astype(int)

# print(train_final_df.loc[422, 'exam_numeric'])
# print(f"Number of NaN values in 'Age' column: {train_final_df['Answer_Text'].isna().sum()}")

input_ids_ques, attention_masks_ques = encode_data(tokenizer, train_final_df['ItemStem_Text'].values)
input_ids_answers, attention_masks_answers = encode_data(tokenizer, train_final_df['combined_answer_text'].values)
input_ids_correct_ans, attention_masks_correct_ans = encode_data(tokenizer, train_final_df['Answer_Text'].values)
# print(input_ids_list.size())
readability_scores = torch.tensor(train_final_df['readability_score'].values).unsqueeze(1)
ques_type = torch.tensor(train_final_df['type_numeric'].values).unsqueeze(1)
exam_type = torch.tensor(train_final_df['exam_numeric'].values).unsqueeze(1)
response_time = torch.tensor(train_final_df['Response_Time'].values).unsqueeze(1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pa

In [ ]:
from transformers import BertModel
import torch.nn as nn


class QuestionRespnseTimeModel(nn.Module):
    def __init__(self):
        super(QuestionRespnseTimeModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.regression = nn.Sequential(
            nn.Linear(768 + 768 + 768 + 1 + 1 + 1, 128),  # Combine BERT output and readability_score
            nn.ReLU(),
            nn.Dropout(0.1),  # Existing dropout layer
            nn.Linear(128, 64),  # Additional layer for more complexity
            nn.ReLU(),
            nn.Dropout(0.2),  # Additional dropout layer for regularization
            nn.Linear(64, 1)
        )

    def forward(self, input_ids_ques, input_ids_answers, input_ids_correct_ans, attention_masks_ques, attention_masks_answers, attention_masks_correct_ans, readability_scores, exam_type, ques_type):
        bert_output_ques = self.bert(input_ids=input_ids_ques, attention_mask=attention_masks_ques)
        pooled_output_ques = bert_output_ques.pooler_output

        bert_output_answers = self.bert(input_ids=input_ids_answers, attention_mask=attention_masks_ques)
        pooled_output_answers = bert_output_answers.pooler_output

        bert_output_correct_ans = self.bert(input_ids=input_ids_correct_ans, attention_mask=attention_masks_correct_ans)
        pooled_output_correct_ans = bert_output_correct_ans.pooler_output

        combined_features = torch.cat((pooled_output_ques, pooled_output_answers, pooled_output_correct_ans, readability_scores, exam_type, ques_type), dim=1)
        return self.regression(combined_features)

In [ ]:
from torch.utils.data import random_split

dataset_size = len(train_final_df)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size

print("Sizes:")
print(input_ids_ques.size())
print(attention_masks_correct_ans.size())
print(readability_scores.size())
print(exam_type.size())
print(ques_type.size())
print(response_time.size())

# full_dataset = TensorDataset(input_ids_list[0], input_ids_list[1], input_ids_list[2], attention_masks_list[0], attention_masks_list[1], attention_masks_list[2], readability_scores, exam_type, ques_type, difficulties)
full_dataset = TensorDataset(input_ids_ques, input_ids_answers, input_ids_correct_ans, attention_masks_ques, attention_masks_answers, attention_masks_correct_ans, readability_scores, exam_type, ques_type, response_time)
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

batch_size = 4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

Sizes:
torch.Size([466, 512])
torch.Size([466, 512])
torch.Size([466, 1])
torch.Size([466, 1])
torch.Size([466, 1])
torch.Size([466, 1])


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = QuestionRespnseTimeModel().to(device)
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
criterion = nn.MSELoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
epochs = 30
best_val_loss = float('inf')
patience = 7
counter = 0

for epoch in range(epochs):
    model.train()
    total_train_loss = 0
    print(f"Epoch {epoch} :")
    for batch in train_dataloader:
        batch = tuple(b.to(device) for b in batch)
        input_ques, input_answers, input_correct_ans, mask_ques, mask_answers, mask_correct_ans, readability, exam, ques, responseTime = batch
        input_ques = input_ques.to(device).long()
        input_answers = input_answers.to(device).long()
        input_correct_ans = input_correct_ans.to(device).long()
        mask_ques = mask_ques.to(device).long()
        mask_answers = mask_answers.to(device).long()
        mask_correct_ans = mask_correct_ans.to(device).long()
        # inputs = list(zip(input_ques, input_answers, input_correct_ans)).to(device).long()
        # masks = list(zip(input_ques, input_answers, input_correct_ans)).to(device).long()
        # inputs, masks, readability, exam_type, ques_types, difficulty = batch
        # inputs = inputs.to(device).long()
        # masks = masks.to(device).long()

        readability = readability.to(device).float()
        exam = exam.to(device).float()
        ques = ques.to(device).float()
        responseTime = responseTime.float()
        model.zero_grad()
        outputs = model( input_ques, input_answers, input_correct_ans, mask_ques, mask_answers, mask_correct_ans, readability, exam,ques )
        loss = criterion(outputs.squeeze(), responseTime)
        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            batch = tuple(b.to(device) for b in batch)
            # input_ques, input_answers, input_correct_ans, mask_ques, mask_answers, mask_correct_ans, readability, exam_type, ques_types, difficulty = batch
            # inputs = list(zip(input_ques, input_answers, input_correct_ans)).to(device).long()
            # masks = list(zip(input_ques, input_answers, input_correct_ans)).to(device).long()
            input_ques, input_answers, input_correct_ans, mask_ques, mask_answers, mask_correct_ans, readability, exam, ques, responseTime = batch
            input_ques = input_ques.to(device).long()
            input_answers = input_answers.to(device).long()
            input_correct_ans = input_correct_ans.to(device).long()
            mask_ques = mask_ques.to(device).long()
            mask_answers = mask_answers.to(device).long()
            mask_correct_ans = mask_correct_ans.to(device).long()
            readability = readability.float()
            exam = exam.to(device).float()
            ques = ques.to(device).float()
            responseTime = responseTime.float()
            outputs = model( input_ques, input_answers, input_correct_ans, mask_ques, mask_answers, mask_correct_ans, readability, exam, ques)
            loss = criterion(outputs, responseTime)
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_dataloader)
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), 'best_model_response_time.pth')
        counter = 0  # Reset the counter if validation loss improves
    else:
        counter += 1  # Increment the counter if validation loss doesn't improve
        if counter >= patience:
            print(f'Early stopping! No improvement in validation loss for {patience} epochs.')
            break  # Exit the training loop
    print(f"Epoch {epoch+1}: Avg Val Loss: {avg_val_loss:.2f}")

Epoch 0 :


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4, 1])) that is different to the input size (torch.Size([4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1: Avg Val Loss: 6626.99
Epoch 1 :
Epoch 2: Avg Val Loss: 5464.82
Epoch 2 :
Epoch 3: Avg Val Loss: 3912.21
Epoch 3 :
Epoch 4: Avg Val Loss: 2416.81
Epoch 4 :
Epoch 5: Avg Val Loss: 1326.66
Epoch 5 :
Epoch 6: Avg Val Loss: 749.24
Epoch 6 :
Epoch 7: Avg Val Loss: 575.27
Epoch 7 :
Epoch 8: Avg Val Loss: 556.42
Epoch 8 :
Epoch 9: Avg Val Loss: 567.59
Epoch 9 :
Epoch 10: Avg Val Loss: 572.28
Epoch 10 :
Early stopping! No improvement in validation loss for 3 epochs.


In [ ]:
!scp -r /content/best_model_response_time.pth '/content/gdrive/My Drive/ML-Project/'

In [ ]:
# Assuming test_df is your test dataset loaded and preprocessed similarly
test_final_df = pd.read_csv(test_file)
# input_ids_test, attention_masks_test = encode_data(tokenizer, test_df['ItemStem_Text'].values)

# test_df['readability_score'] = test_df['ItemStem_Text'].apply(preprocess_text)
# test_df[answer_columns] = ' ' + test_df[answer_columns].fillna('')

# # Combine itemtext and answer columns into 'combined_text' column
# test_df['combined_answer_text'] = test_df[answer_columns].agg(' '.join, axis=1)
# test_df['combined_answer_text'] = test_df['combined_answer_text'].astype(str)
# # print(train_final_df.loc[0, 'combined_text'])

# test_df['Answer_Text'] = test_df['Answer_Text'].fillna('')
# test_df['type_numeric'] = test_df['ItemType'].replace(type_mapping).astype(int)
# test_df['exam_numeric'] = test_df['EXAM'].replace(exam_mapping).astype(int)

# input_ids_ques_test, attention_masks_ques_test = encode_data(tokenizer, test_df['ItemStem_Text'].values)
# input_ids_answers_test, attention_masks_answers_test = encode_data(tokenizer, test_df['combined_answer_text'].values)
# input_ids_correct_ans_test, attention_masks_correct_ans_test = encode_data(tokenizer, test_df['Answer_Text'].values)

# readability_scores_test = torch.tensor(test_df['readability_score'].values).unsqueeze(1)
# ques_type_test = torch.tensor(test_df['type_numeric'].values).unsqueeze(1)
# exam_type_test = torch.tensor(test_df['exam_numeric'].values).unsqueeze(1)



# Replace NaN values with empty strings in answer columns to avoid NaN in combined text
answer_columns = ['Answer__A', 'Answer__B', 'Answer__C', 'Answer__D', 'Answer__E',
                  'Answer__F', 'Answer__G', 'Answer__H', 'Answer__I', 'Answer__J']
test_final_df[answer_columns] = ' ' + test_final_df[answer_columns].fillna('')

test_final_df['readability_score'] = test_final_df['ItemStem_Text'].apply(preprocess_text)


# Combine itemtext and answer columns into 'combined_text' column
test_final_df['combined_answer_text'] = test_final_df[answer_columns].agg(' '.join, axis=1)
test_final_df['combined_answer_text'] = test_final_df['combined_answer_text'].astype(str)
# print(test_final_df.loc[0, 'combined_text'])

test_final_df['Answer_Text'] = test_final_df['Answer_Text'].fillna('')

# Convert 'type' column to numerical values
type_mapping = {'Text': 0, 'PIX': 1}
test_final_df['type_numeric'] = test_final_df['ItemType'].replace(type_mapping).astype(int)

# Convert 'exam' column to numerical values
exam_mapping = {'STEP 1': 0, 'STEP 2': 1, 'STEP 3': 2}
test_final_df['exam_numeric'] = test_final_df['EXAM'].replace(exam_mapping).astype(int)

# print(test_final_df.loc[422, 'exam_numeric'])
# print(f"Number of NaN values in 'Age' column: {test_final_df['Answer_Text'].isna().sum()}")

input_ids_ques_test, attention_masks_ques_test = encode_data(tokenizer, test_final_df['ItemStem_Text'].values)
input_ids_answers_test, attention_masks_answers_test = encode_data(tokenizer, test_final_df['combined_answer_text'].values)
input_ids_correct_ans_test, attention_masks_correct_ans_test = encode_data(tokenizer, test_final_df['Answer_Text'].values)
# print(input_ids_list.size())
readability_scores = torch.tensor(test_final_df['readability_score'].values).unsqueeze(1)
ques_type = torch.tensor(test_final_df['type_numeric'].values).unsqueeze(1)
exam_type = torch.tensor(test_final_df['exam_numeric'].values).unsqueeze(1)
# difficulties = torch.tensor(test_final_df['Difficulty'].values).unsqueeze(1)

test_dataset = TensorDataset(input_ids_ques_test, input_ids_answers_test, input_ids_correct_ans_test, attention_masks_ques_test, attention_masks_answers_test, attention_masks_correct_ans_test, readability_scores, exam_type, ques_type)
test_dataloader = DataLoader(test_dataset, batch_size=4)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
model = QuestionRespnseTimeModel().to(device)
model.load_state_dict(torch.load('best_model_response_time.pth'))
model.eval()

QuestionRespnseTimeModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [ ]:
predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ques, input_answers, input_correct_ans, mask_ques, mask_answers, mask_correct_ans, readability, exam_type, ques_types = [b.to(device) for b in batch]
        input_ques = input_ques.to(device).long()
        input_answers = input_answers.to(device).long()
        input_correct_ans = input_correct_ans.to(device).long()
        mask_ques = mask_ques.to(device).long()
        mask_answers = mask_answers.to(device).long()
        mask_correct_ans = mask_correct_ans.to(device).long()
        readability = readability.float()
        exam_type = exam_type.to(device).float()
        ques_types = ques_types.to(device).float()
        outputs = model( input_ques, input_answers, input_correct_ans, mask_ques, mask_answers, mask_correct_ans, readability, exam_type, ques_types)
        predictions.extend(outputs.cpu().numpy())

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
# Assuming 'Difficulty' is the column in test_df with true difficulty scores
gold_df = pd.read_csv(gold_file)
true_response_time =[]
itemNums = test_final_df['ItemNum'].values
for item in itemNums:
    true_response_time.append(gold_df.loc[gold_df['ItemNum'] == item , 'Response_Time'])

rmse = sqrt(mean_squared_error(true_response_time, predictions))
print(f"RMSE on Test Data on Response Time: {rmse}")

RMSE on Test Data on Response Time: 25.96172976849861
